### Importing

In [1]:
import sys
import pandas as pd 
import numpy as np
#np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import plotly.express as px
%matplotlib inline

# Extracting data from single folder (YAW)

### Path

In [2]:
%%time
os.chdir(r'../data/raw/Yaw-01/')
all_files=glob.glob('*int')
print(all_files)

['0004.int', '0005.int', '0002.int', '0001.int', '0003.int']
CPU times: user 466 µs, sys: 182 µs, total: 648 µs
Wall time: 1.33 ms


In [4]:
# os.getcwd()

### Function of Reading binary data

In [3]:
sensor = (1,4,5,6,7,8,9,10,11,12,53,54,56,57,59,60,61,62,63,64)



def read_int(FileName,x):

    #print(FileName)

    xlen = len(FileName)
    ending = FileName[xlen-4:xlen]

    if ending=='.int':
        FileName = FileName[0:xlen-4]


    with open(FileName+ending,"rb") as f:
        f.read()
        filesize = f.tell()
       # print(filesize)
       

    f.close()




    with open(FileName+ending,"rb") as f:

    # the first 4 junk buffer
        
        leer = np.frombuffer(f.read(4),dtype=np.single)
        #print(leer)
    # the  24 buffer give information about file
        datei = np.frombuffer(f.read(24),dtype=np.uint32)
        #print(datei)

        f.seek(76)
    # read the number of sensor in file 
        lvek = np.frombuffer(f.read(4),dtype=np.uint32)
        #print(lvek)
     
    # then 4 times the number of sensors are skipped to read a control parameter:
  
        f.seek(84+lvek[0]*4)

        filetype = np.frombuffer(f.read(4),dtype=np.uint)
        #print(filetype)

        if filetype != 12:

    # start time is read (is not equal to 0 as we have been cut off from the simulation for the first 50 s
            t0 = np.frombuffer(f.read(4),dtype=np.single)
            #print(t0)

    # comes the parameter for the step size of the simulation

            dt = np.frombuffer(f.read(4),dtype=np.single)
            #print(dt)

            pos_fak = f.tell()
            #print(pos_fak)

            fak = []
 
            for n in x:
                f.seek(pos_fak + 4*(n-1))
                fak.append(np.frombuffer(f.read(4),dtype=np.single))
                
            f.seek(pos_fak+lvek[0]*4)
            position = f.tell()
    # the length of the sensors is calculated:
            RecordCount = round((filesize-position)/lvek[0]/2)
            #print(RecordCount)

            raw_ts_int = np.empty([len(x),RecordCount])
           # print(raw_ts_int)
        
            #print("Type:",type(raw_ts_int))
            #print(raw_ts_int.shape)

            for n in range(RecordCount):

                for m, sensor_ in enumerate(x):

                    f.seek(position+2*sensor_+(lvek[0]*2)*n-2)
                    raw_ts_int[m,n] = np.frombuffer(f.read(2),dtype=np.int16)*fak[m]

            global   df
            df = pd.DataFrame(raw_ts_int)

            #print(df.transpose())
            #merged_df=df.merge(df)
                    
    f.close()

### Sample of single file dataset

In [4]:
%%time
read_int('0001.int',sensor)
raw_1stfile=df.transpose()
raw_1stfile

CPU times: total: 1.58 s
Wall time: 1.6 s


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3.880149,-0.515813,-1.311453e-12,-6.348142e-13,-4.441732e-13,350.412323,608.005310,7.645635,0.887514,56.532722,-0.080608,0.023487,0.021482,0.039993,0.697733,0.852825,0.809038,-0.150583,0.327039,-0.400013
1,3.935400,-0.606380,-1.314330e-12,-6.373553e-13,-4.465209e-13,351.784821,607.934387,7.645635,0.882614,56.189407,-0.082640,0.027315,0.021740,0.039731,0.694794,0.848931,0.812124,-0.141381,0.325328,-0.406860
2,3.943913,-0.637350,-1.317104e-12,-6.397469e-13,-4.486338e-13,353.157288,607.898926,7.646081,0.877223,55.846092,-0.083835,0.037420,0.021925,0.039501,0.691047,0.845919,0.815209,-0.132299,0.323432,-0.413388
3,3.982914,-0.561877,-1.319981e-12,-6.421385e-13,-4.509814e-13,354.541046,607.863403,7.645189,0.873302,55.617214,-0.080767,0.016163,0.022036,0.039296,0.686712,0.844229,0.817928,-0.123217,0.321232,-0.419668
4,3.966663,-0.897598,-1.322961e-12,-6.446795e-13,-4.533290e-13,355.913513,607.792480,7.644742,0.867422,55.216679,-0.081772,0.023983,0.022073,0.039108,0.681717,0.843789,0.820132,-0.114135,0.318572,-0.425805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,4.421057,-0.019258,-3.105234e-12,4.780912e-12,7.511035e-12,275.364136,609.388611,7.666604,1.039926,66.374451,-0.003470,0.031307,-0.004812,0.052105,0.662247,0.701333,0.845331,-0.506528,0.169648,0.100864
19997,4.386854,-0.232402,-3.106261e-12,4.780315e-12,7.510565e-12,276.747894,609.424072,7.666158,1.042866,66.546112,-0.002190,-0.000385,-0.004966,0.052589,0.665994,0.701259,0.852164,-0.504371,0.176037,0.092526
19998,4.415331,0.125180,-3.107289e-12,4.779567e-12,7.510096e-12,278.131622,609.388611,7.666158,1.042866,66.546112,-0.000169,0.008894,-0.005124,0.053073,0.669227,0.701920,0.857748,-0.502095,0.182150,0.084153
19999,4.403569,0.553289,-3.108419e-12,4.778820e-12,7.509627e-12,279.504120,609.424072,7.667050,1.042866,66.546112,0.001894,0.019935,-0.005280,0.053573,0.671798,0.703610,0.862082,-0.499650,0.188233,0.075781


### Extracting required dataframe from raw data

In [5]:
%%time
file_list = glob.glob("*.int")

mean_df = pd.DataFrame()
min_df = pd.DataFrame()
max_df = pd.DataFrame()
median_df = pd.DataFrame()
std_df = pd.DataFrame()
q1_df = pd.DataFrame()
q3_df = pd.DataFrame()

for file_path in file_list:
   
    read_int(file_path, sensor)
    data = df.transpose()

    mean_df = pd.concat([mean_df, data.mean()], axis=1)
    min_df = pd.concat([min_df, data.min()], axis=1)
    max_df = pd.concat([max_df, data.max()], axis=1)
    median_df = pd.concat([median_df, data.median()], axis=1)
    std_df = pd.concat([std_df, data.std()], axis=1)
    q1_df = pd.concat([q1_df, data.quantile(0.25)], axis=1)
    q3_df = pd.concat([q3_df, data.quantile(0.75)], axis=1)

mean_df = mean_df.transpose()
min_df = min_df.transpose()
max_df = max_df.transpose()
median_df = median_df.transpose()
std_df = std_df.transpose()
q1_df = q1_df.transpose()
q3_df = q3_df.transpose()

mean_df.reset_index(drop=True, inplace=True)
min_df.reset_index(drop=True, inplace=True)
max_df.reset_index(drop=True, inplace=True)
median_df.reset_index(drop=True, inplace=True)
std_df.reset_index(drop=True, inplace=True)
q1_df.reset_index(drop=True, inplace=True)
q3_df.reset_index(drop=True, inplace=True)

print("Mean:\n", mean_df)
print("Min:\n", min_df)
print("Max:\n", max_df)
print("Median:\n", median_df)
print("Standard Deviation:\n", std_df)
print("Q1:\n", q1_df)
print("Q3:\n", q3_df)

Mean:
              0         1             2             3             4  \
0     3.972564  0.914831 -2.254247e-12  2.019298e-12  3.487172e-12   
1     3.972564  0.914831 -2.254247e-12  2.019298e-12  3.487172e-12   
2     3.972564  0.914831 -2.254247e-12  2.019298e-12  3.487172e-12   
3     4.010808  0.779758 -2.252037e-12  2.007514e-12  3.480714e-12   
4     4.514270  1.018148 -2.321417e-12  2.094770e-12  3.575547e-12   
..         ...       ...           ...           ...           ...   
245  12.975750  1.059399  9.226112e+00  9.226112e+00  9.226112e+00   
246  13.050791  0.975248  9.387131e+00  9.387131e+00  9.387131e+00   
247  12.995100  0.985781  9.269344e+00  9.269344e+00  9.269344e+00   
248  13.429026  0.987961  1.025056e+01  1.025056e+01  1.025056e+01   
249  13.475914  1.059149  1.026892e+01  1.026892e+01  1.026892e+01   

              5            6          7          8            9        10  \
0    179.659560   610.613874   7.680688   1.042556    66.767276 -0.000027  

In [6]:
mean_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3.972564,0.914831,-2.254247e-12,2.019298e-12,3.487172e-12,179.659560,610.613874,7.680688,1.042556,66.767276,-0.000027,0.000046,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448
1,3.972564,0.914831,-2.254247e-12,2.019298e-12,3.487172e-12,179.659560,610.613874,7.680688,1.042556,66.767276,-0.000027,0.000046,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448
2,3.972564,0.914831,-2.254247e-12,2.019298e-12,3.487172e-12,179.659560,610.613874,7.680688,1.042556,66.767276,-0.000027,0.000046,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448
3,4.010808,0.779758,-2.252037e-12,2.007514e-12,3.480714e-12,179.787905,611.051802,7.686194,1.054805,67.629964,-0.000023,0.000019,-0.004680,0.057457,0.727423,0.727231,0.726922,-0.078315,-0.079563,-0.079115
4,4.514270,1.018148,-2.321417e-12,2.094770e-12,3.575547e-12,179.869600,633.038380,7.962755,1.950605,129.550192,-0.000022,0.000045,-0.006732,0.084048,0.505390,0.505410,0.504651,-0.097841,-0.100096,-0.097765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,12.975750,1.059399,9.226112e+00,9.226112e+00,9.226112e+00,179.791322,1149.842938,14.463432,17.026750,2050.000977,0.000013,0.000089,-0.042049,0.371709,-1.151974,-1.148972,-1.148519,-0.098579,-0.099675,-0.098825
246,13.050791,0.975248,9.387131e+00,9.387131e+00,9.387131e+00,179.764973,1150.090929,14.466560,17.022417,2049.948556,-0.000009,0.000179,-0.041072,0.367600,-1.111452,-1.110045,-1.111690,-0.094134,-0.095431,-0.094784
247,12.995100,0.985781,9.269344e+00,9.269344e+00,9.269344e+00,179.805280,1150.100268,14.466679,17.023052,2050.039037,-0.000064,-0.000096,-0.041202,0.370525,-1.148464,-1.146829,-1.146034,-0.097859,-0.099069,-0.098056
248,13.429026,0.987961,1.025056e+01,1.025056e+01,1.025056e+01,179.614961,1150.109982,14.466800,17.022466,2050.017880,0.000056,0.000306,-0.041945,0.354075,-0.931933,-0.929262,-0.933211,-0.075254,-0.076005,-0.075551


In [7]:
column_names = {
    int('0'): 'wind_speed',
    int('1'): 'yaw_error',
    int('2'): 'pitch_angle_b1',
    int('3'): 'pitch_angle_b2',
    int('4'): 'pitch_angle_b3',
    int('5'): 'rotorposition',
    int('6'): 'generator_speed',
    int('7'): 'rotor_speed',
    int('8'): 'generator_torque',
    int('9'): 'power',
    int('10'): 'acceleration_CROSS',
    int('11'): 'acceleration_THRUST',
    int('12'): 'tower_deflection_Y',
    int('13'): 'tower_deflection_Z',
    int('14'): 'tip_deflection_flap_V1',
    int('15'): 'tip_deflection_flap_V2',
    int('16'): 'tip_deflection_flap_V3',
    int('17'): 'tip_deflection_edge_V1',
    int('18'): 'tip_deflection_edge_V2',
    int('19'): 'tip_deflection_edge_V3',
}

for df_name in ['mean', 'max', 'min', 'median', 'std', 'q1', 'q3']:
    df = globals()[df_name + '_df']
    df.rename(columns=column_names, inplace=True)
    print(f'{df_name}:')
    print(df)
    df.to_parquet(df_name + '.parquet')


mean:
     wind_speed  yaw_error  pitch_angle_b1  pitch_angle_b2  pitch_angle_b3  \
0      3.972564   0.914831   -2.254247e-12    2.019298e-12    3.487172e-12   
1      3.972564   0.914831   -2.254247e-12    2.019298e-12    3.487172e-12   
2      3.972564   0.914831   -2.254247e-12    2.019298e-12    3.487172e-12   
3      4.010808   0.779758   -2.252037e-12    2.007514e-12    3.480714e-12   
4      4.514270   1.018148   -2.321417e-12    2.094770e-12    3.575547e-12   
..          ...        ...             ...             ...             ...   
245   12.975750   1.059399    9.226112e+00    9.226112e+00    9.226112e+00   
246   13.050791   0.975248    9.387131e+00    9.387131e+00    9.387131e+00   
247   12.995100   0.985781    9.269344e+00    9.269344e+00    9.269344e+00   
248   13.429026   0.987961    1.025056e+01    1.025056e+01    1.025056e+01   
249   13.475914   1.059149    1.026892e+01    1.026892e+01    1.026892e+01   

     rotorposition  generator_speed  rotor_speed  generat

### Calulating the turbulence, Turbulence(Label)

In [8]:
turbulence = std_df['wind_speed'] / mean_df['wind_speed']

mean_df['turbulence'] = turbulence
mean_df

,wind_speed,yaw_error,pitch_angle_b1,pitch_angle_b2,pitch_angle_b3,rotorposition,generator_speed,rotor_speed,generator_torque,power,...,acceleration_THRUST,tower_deflection_Y,tower_deflection_Z,tip_deflection_flap_V1,tip_deflection_flap_V2,tip_deflection_flap_V3,tip_deflection_edge_V1,tip_deflection_edge_V2,tip_deflection_edge_V3,turbulence
0,3.972564,0.914831,-2.254247e-12,2.019298e-12,3.487172e-12,179.659560,610.613874,7.680688,1.042556,66.767276,...,0.000046,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448,0.066145
1,3.972564,0.914831,-2.254247e-12,2.019298e-12,3.487172e-12,179.659560,610.613874,7.680688,1.042556,66.767276,...,0.000046,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448,0.066145
2,3.972564,0.914831,-2.254247e-12,2.019298e-12,3.487172e-12,179.659560,610.613874,7.680688,1.042556,66.767276,...,0.000046,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448,0.066145
3,4.010808,0.779758,-2.252037e-12,2.007514e-12,3.480714e-12,179.787905,611.051802,7.686194,1.054805,67.629964,...,0.000019,-0.004680,0.057457,0.727423,0.727231,0.726922,-0.078315,-0.079563,-0.079115,0.075750
4,4.514270,1.018148,-2.321417e-12,2.094770e-12,3.575547e-12,179.869600,633.038380,7.962755,1.950605,129.550192,...,0.000045,-0.006732,0.084048,0.505390,0.505410,0.504651,-0.097841,-0.100096,-0.097765,0.067105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,12.975750,1.059399,9.226112e+00,9.226112e+00,9.226112e+00,179.791322,1149.842938,14.463432,17.026750,2050.000977,...,0.000089,-0.042049,0.371709,-1.151974,-1.148972,-1.148519,-0.098579,-0.099675,-0.098825,0.082844
246,13.050791,0.975248,9.387131e+00,9.387131e+00,9.387131e+00,179.764973,1150.090929,14.466560,17.022417,2049.948556,...,0.000179,-0.041072,0.367600,-1.111452,-1.110045,-1.111690,-0.094134,-0.095431,-0.094784,0.079974
247,12.995100,0.985781,9.269344e+00,9.269344e+00,9.269344e+00,179.805280,1150.100268,14.466679,17.023052,2050.039037,...,-0.000096,-0.041202,0.370525,-1.148464,-1.146829,-1.146034,-0.097859,-0.099069,-0.098056,0.082719
248,13.429026,0.987961,1.025056e+01,1.025056e+01,1.025056e+01,179.614961,1150.109982,14.466800,17.022466,2050.017880,...,0.000306,-0.041945,0.354075,-0.931933,-0.929262,-0.933211,-0.075254,-0.076005,-0.075551,0.080598


In [9]:
q3_df

,wind_speed,yaw_error,pitch_angle_b1,pitch_angle_b2,pitch_angle_b3,rotorposition,generator_speed,rotor_speed,generator_torque,power,acceleration_CROSS,acceleration_THRUST,tower_deflection_Y,tower_deflection_Z,tip_deflection_flap_V1,tip_deflection_flap_V2,tip_deflection_flap_V3,tip_deflection_edge_V1,tip_deflection_edge_V2,tip_deflection_edge_V3
0,4.154085,2.348746,-1.788541e-12,3.350600e-12,5.445583e-12,269.390442,612.509827,7.704973,1.203118,77.188911,0.005184,0.020459,-0.003267,0.064111,0.812711,0.815870,0.816238,0.223095,0.220604,0.222801
1,4.154085,2.348746,-1.788541e-12,3.350600e-12,5.445583e-12,269.390442,612.509827,7.704973,1.203118,77.188911,0.005184,0.020459,-0.003267,0.064111,0.812711,0.815870,0.816238,0.223095,0.220604,0.222801
2,4.154085,2.348746,-1.788541e-12,3.350600e-12,5.445583e-12,269.390442,612.509827,7.704973,1.203118,77.188911,0.005184,0.020459,-0.003267,0.064111,0.812711,0.815870,0.816238,0.223095,0.220604,0.222801
3,4.210862,2.123348,-1.774643e-12,3.339615e-12,5.445981e-12,269.480103,613.112793,7.712557,1.225172,78.675667,0.006092,0.018949,-0.002800,0.065045,0.811977,0.811977,0.810507,0.222645,0.221802,0.222708
4,4.701898,2.565332,-1.847968e-12,3.474770e-12,5.585756e-12,269.287720,640.104553,8.051629,2.200408,147.511536,0.009578,0.021699,-0.004171,0.091929,0.598844,0.597374,0.595685,0.205844,0.201973,0.205048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,13.750338,2.914706,1.059618e+01,1.059618e+01,1.059618e+01,269.557617,1157.666260,14.562039,17.136126,2056.058350,0.039513,0.068134,-0.029678,0.408669,-0.710017,-0.727362,-0.705459,0.194123,0.192827,0.194130
246,13.743284,2.709858,1.048181e+01,1.048181e+01,1.048181e+01,269.553711,1158.774292,14.575916,17.121792,2055.788818,0.041060,0.070128,-0.029436,0.402327,-0.731216,-0.695545,-0.705353,0.197801,0.194579,0.195086
247,13.747210,2.845974,1.059148e+01,1.059148e+01,1.059148e+01,269.657043,1158.164429,14.568141,17.121061,2055.889648,0.045987,0.066643,-0.026298,0.406344,-0.750805,-0.738221,-0.733218,0.191710,0.190529,0.192557
248,14.216485,2.724953,1.149008e+01,1.149008e+01,1.149008e+01,269.353668,1157.602051,14.559908,17.114996,2055.282471,0.041114,0.070628,-0.029314,0.388813,-0.535318,-0.539197,-0.529646,0.219143,0.217431,0.218272


### Creating Input dataframe

In [11]:
column_names = ['pitch_angle_b1', 'pitch_angle_b2', 'pitch_angle_b3', 'rotorposition', 
                'generator_speed', 'rotor_speed', 'generator_torque', 'power', 
                'acceleration_CROSS', 'acceleration_THRUST']

column_dfs = {}

for column in column_names:
    column_df = pd.DataFrame({'mean': mean_df[column], 
                              'max': max_df[column], 
                              'min': min_df[column], 
                              'median': median_df[column], 
                              'std': std_df[column], 
                              'q1': q1_df[column], 
                              'q3': q3_df[column]})
    
    column_dfs[column] = column_df.rename(columns={
        'mean': f'{column}_mean', 
        'max': f'{column}_max', 
        'min': f'{column}_min', 
        'median': f'{column}_median', 
        'std': f'{column}_std', 
        'q1': f'{column}_q1', 
        'q3': f'{column}_q3'
    })

inputs_df = pd.concat(column_dfs.values(), axis=1)
#inputs_df.to_parquet('inputs.parquet')


In [12]:
inputs_df

,pitch_angle_b1_mean,pitch_angle_b1_max,pitch_angle_b1_min,pitch_angle_b1_median,pitch_angle_b1_std,pitch_angle_b1_q1,pitch_angle_b1_q3,pitch_angle_b2_mean,pitch_angle_b2_max,pitch_angle_b2_min,...,acceleration_CROSS_std,acceleration_CROSS_q1,acceleration_CROSS_q3,acceleration_THRUST_mean,acceleration_THRUST_max,acceleration_THRUST_min,acceleration_THRUST_median,acceleration_THRUST_std,acceleration_THRUST_q1,acceleration_THRUST_q3
0,-2.254247e-12,-1.239936e-12,-3.288136e-12,-2.253504e-12,5.494449e-13,-2.728537e-12,-1.788541e-12,2.019298e-12,4.783154e-12,-7.291321e-13,...,0.016052,-0.005163,0.005184,0.000046,0.117767,-0.116638,0.000303,0.029811,-0.020046,0.020459
1,-2.254247e-12,-1.239936e-12,-3.288136e-12,-2.253504e-12,5.494449e-13,-2.728537e-12,-1.788541e-12,2.019298e-12,4.783154e-12,-7.291321e-13,...,0.016052,-0.005163,0.005184,0.000046,0.117767,-0.116638,0.000303,0.029811,-0.020046,0.020459
2,-2.254247e-12,-1.239936e-12,-3.288136e-12,-2.253504e-12,5.494449e-13,-2.728537e-12,-1.788541e-12,2.019298e-12,4.783154e-12,-7.291321e-13,...,0.016052,-0.005163,0.005184,0.000046,0.117767,-0.116638,0.000303,0.029811,-0.020046,0.020459
3,-2.252037e-12,-1.256854e-12,-3.298018e-12,-2.244919e-12,5.510169e-13,-2.725709e-12,-1.774643e-12,2.007514e-12,4.767260e-12,-7.377335e-13,...,0.016339,-0.006113,0.006092,0.000019,0.096923,-0.102976,-0.000255,0.027987,-0.018666,0.018949
4,-2.321417e-12,-1.264833e-12,-3.380494e-12,-2.325040e-12,5.694004e-13,-2.793344e-12,-1.847968e-12,2.094770e-12,4.968172e-12,-7.357552e-13,...,0.022268,-0.009902,0.009578,0.000045,0.115055,-0.130804,0.000194,0.032029,-0.021755,0.021699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,9.226112e+00,1.315070e+01,2.922546e+00,9.367576e+00,1.785962e+00,7.958986e+00,1.059618e+01,9.226112e+00,1.315070e+01,2.922546e+00,...,0.055363,-0.038926,0.039513,0.000089,0.384183,-0.370951,0.001209,0.101661,-0.068516,0.068134
246,9.387131e+00,1.424340e+01,2.496601e+00,9.485661e+00,1.775417e+00,8.473043e+00,1.048181e+01,9.387131e+00,1.424340e+01,2.496601e+00,...,0.058794,-0.040951,0.041060,0.000179,0.434520,-0.371732,-0.000062,0.102217,-0.068031,0.070128
247,9.269344e+00,1.427483e+01,2.887977e+00,9.379456e+00,1.861192e+00,8.040298e+00,1.059148e+01,9.269344e+00,1.427483e+01,2.887977e+00,...,0.064626,-0.046096,0.045987,-0.000096,0.427306,-0.375574,0.000974,0.098808,-0.067130,0.066643
248,1.025056e+01,1.411192e+01,3.268729e+00,1.035347e+01,1.692245e+00,9.252270e+00,1.149008e+01,1.025056e+01,1.411192e+01,3.268729e+00,...,0.059022,-0.040765,0.041114,0.000306,0.357054,-0.361880,0.000391,0.104681,-0.071150,0.070628


### Creating output dataframe

In [13]:
targets_df = mean_df[['wind_speed', 'yaw_error', 'turbulence','tower_deflection_Y', 'tower_deflection_Z', 'tip_deflection_flap_V1', 'tip_deflection_flap_V2', 'tip_deflection_flap_V3', 'tip_deflection_edge_V1','tip_deflection_edge_V2','tip_deflection_edge_V3']].copy()
#targets_df.to_parquet('targets.parquet')

In [14]:
targets_df

,wind_speed,yaw_error,turbulence,tower_deflection_Y,tower_deflection_Z,tip_deflection_flap_V1,tip_deflection_flap_V2,tip_deflection_flap_V3,tip_deflection_edge_V1,tip_deflection_edge_V2,tip_deflection_edge_V3
0,3.972564,0.914831,0.066145,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448
1,3.972564,0.914831,0.066145,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448
2,3.972564,0.914831,0.066145,-0.004686,0.057184,0.730748,0.729019,0.730578,-0.078014,-0.079855,-0.078448
3,4.010808,0.779758,0.075750,-0.004680,0.057457,0.727423,0.727231,0.726922,-0.078315,-0.079563,-0.079115
4,4.514270,1.018148,0.067105,-0.006732,0.084048,0.505390,0.505410,0.504651,-0.097841,-0.100096,-0.097765
...,...,...,...,...,...,...,...,...,...,...,...
245,12.975750,1.059399,0.082844,-0.042049,0.371709,-1.151974,-1.148972,-1.148519,-0.098579,-0.099675,-0.098825
246,13.050791,0.975248,0.079974,-0.041072,0.367600,-1.111452,-1.110045,-1.111690,-0.094134,-0.095431,-0.094784
247,12.995100,0.985781,0.082719,-0.041202,0.370525,-1.148464,-1.146829,-1.146034,-0.097859,-0.099069,-0.098056
248,13.429026,0.987961,0.080598,-0.041945,0.354075,-0.931933,-0.929262,-0.933211,-0.075254,-0.076005,-0.075551
